In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from ipywidgets import GridspecLayout,HTML,Layout,interactive_output,IntSlider,interact, interactive
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
from datetime import date
from datetime import datetime
plt.style.use('dark_background')

grid = GridspecLayout(5, 3)

title1 = widgets.HTML(
#     value="<h1>★★★★★  Parabolic Solar Dish Concentrator Calculator ★★★★★</h1>",
    value=f"<center><font size = 6><b><font color='black'>{'Parabolic Solar Dish Concentrator Calculator'}</b></center>",
)
title2 = widgets.HTML(
#     value="<h0>------------------------------------------- Desarrollado por Aguilar Hancco Junior Joel, estudiante de la carrera de Ingenieria Mecanica ---------------------------------------------------</h0>",
    value=f"<center><font size = 1><b><font color='black'>{'Desarrollado por Aguilar Hancco Junior Joel, estudiante de Ingenieria Mecanica'}</b></center>",
)
title3 = widgets.HTML(
    value=f"<center><font size = 3><b><font color='black'>{'Parametro de calculo '}</b></center>",
)
latitud = widgets.FloatSlider(
    value=-16.41833,
    min=-90,
    max=90,
    step=0.1,
    description='Latitud: ',
    continuous_update=False,
    disabled=False
)
hora = widgets.FloatSlider(
    value=12,
    min=0,
    max=24,
    step=(24 - 0) / ((24 * 60) - 1),
    description='Hora: ',
    continuous_update=False,
    disabled=False
)
altura = widgets.FloatSlider(
    value=2.405,
    min=0,
    max=2.5,
    step=0.1,
    description='M.s.n.m: ',
    continuous_update=False,
    disabled=False
)
longitud = widgets.FloatSlider(
    value=-71.48554,
    min=-180,
    max=180,
    step=0.1,
    description='Longitud: ',
    continuous_update=False,
    disabled=False
)
dia = widgets.IntSlider(
    value=12,
    min=1,
    max=365,
    step=1,
    description='Dia del año: ',
    continuous_update=False,
    disabled=False
)
horas = widgets.FloatRangeSlider(
    value=[0, 24],
    min=0,
    max=24,
    step=0.5,
    description='Radiacion: ',
    continuous_update=False,
    disabled=False
)
latitud.style.handle_color = 'yellow'
hora.style.handle_color = 'yellow'
altura.style.handle_color = 'yellow'
longitud.style.handle_color = 'yellow'
dia.style.handle_color = 'yellow'
horas.style.handle_color = 'yellow'

grid[0,:] = title1
grid[1,:] = title2
grid[2,1] = title3
grid[3,0] = latitud
grid[3,1] = hora
grid[3,2] = dia
grid[4,0] = longitud
grid[4,1] = horas
grid[4,2] = altura

# -------------------------------------------------------------------------------------------------------------------------

grid1 = GridspecLayout(3, 3)

title4 = widgets.HTML(
    value=f"<center><font size = 3><b><font color='black'>{'Resultados'}</b></center>",
)
azimuth = widgets.FloatText(
    value=0,
    description='Azimuth: ',
    continuous_update=False,
    disabled=True
)
altitud = widgets.FloatText(
    value=0,
    description='Altitud: ',
    continuous_update=False,
    disabled=True
)
sunrise = widgets.Text(
    value='',
    description='Sunrise: ',
    continuous_update=False,
    disabled=True
)
sunset = widgets.Text(
    value='',
    description='Sunset: ',
    continuous_update=False,
    disabled=True
)
radiacion = widgets.Text(
    value='',
    description='Radiacion: ',
    continuous_update=False,
    disabled=True
)
duracion = widgets.Text(
    value='',
    description='Duracion: ',
    continuous_update=False,
    disabled=True
)

grid1[0,:] = title4
grid1[1,0] = azimuth
grid1[1,1] = sunrise
grid1[1,2] = radiacion
grid1[2,0] = altitud
grid1[2,1] = sunset
grid1[2,2] = duracion

In [2]:
# -------------------------------------------------------------------------------------------------------------------------

# |------------------------------------------------------------|
# |  Funcion para mantener la relacion de los ejes iguales 3D  |
# |------------------------------------------------------------|
%matplotlib widget

display(grid)

def set_axes_equal(ax):

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

# -------------------------------------------------------------------------------------------------------------------------
    
# |-------------------------------------------------------------|
# |  Grafica del seguimiento del concentrador solar parabolico  |
# |-------------------------------------------------------------|

fig4  = plt.figure(figsize=(18.5,10), dpi= 50, facecolor='black')
fig4.suptitle('Solar Figures')
ax4   = fig4.add_subplot(2, 2, 1, projection='3d')
ax3   = fig4.add_subplot(2, 2, 2)
ax    = fig4.add_subplot(2, 1, 2)

def pdsc(lat,hou,doy):
    
    ax4.clear()
    ndd      = 24 * 60 # Cantidad de angulos de posicion solar a calcular durante el presente dia
    height_z = 24
    radius   = 24
    elements = 30

    # Solar Time
    c0  = ((doy - 1) * 360) / 365
    c1  = 229.2 * (0.000075 + 0.001868 * np.cos(c0 * np.pi/180) - 0.032077 * np.sin(c0 * np.pi/180) - 0.014615 * np.cos(2 * c0 * np.pi/180) - 0.04089 * np.sin(2 * c0 * np.pi/180))
    c2  = (4 * 5 + c1) / 60
    
    hr  = np.linspace(0,24,ndd)
    stp = (24 - 0) / (ndd - 1)
    k   = int((hou / stp + 1) - 1)
    
    Alt = np.zeros(len(hr))
    Azi = np.zeros(len(hr))
    i   = -1

    for h in hr:
        i = i + 1
        w = 15 * (h + c2 - 12) # grados

    # Declination angle
        gam = np.arcsin(0.39795 * np.cos(0.98563 * (doy - 173)  * np.pi/180)) * 180/np.pi

    # Sun altitude angle
        Alt[i] = np.arcsin(np.sin(gam  * np.pi/180) * np.sin(lat  * np.pi/180) + np.cos(gam * np.pi/180) * np.cos(lat * np.pi/180) * np.cos(w * np.pi/180)) * 180/np.pi

    # Azimuth angle
        A   = np.arccos((np.sin(gam  * np.pi/180) * np.cos(lat  * np.pi/180) - np.cos(gam * np.pi/180) * np.sin(lat * np.pi/180) * np.cos(w * np.pi/180)) / np.cos(Alt[i] * np.pi/180)) * 180/np.pi
        if np.sin(w  * np.pi/180) > 0:
            Azi[i] = 360 - A
        else:
            Azi[i] = A
       
    rota_azi = 360 - Azi[k] # rojo zy hora Z
    rota_alt = Alt[k] + 180 # azul xz anti Y
    center_x = height_z * np.cos(rota_alt * np.pi/180) * np.cos(rota_azi * np.pi/180)
    center_y = height_z * np.cos(rota_alt * np.pi/180) * np.sin(rota_azi * np.pi/180)
    center_z = height_z * np.sin(rota_alt * np.pi/180)
    center_X = 5 * height_z * np.cos(Alt[k] * np.pi/180) * np.cos((360 - Azi[k]) * np.pi/180)
    center_Y = 5 * height_z * np.cos(Alt[k] * np.pi/180) * np.sin((360 - Azi[k]) * np.pi/180)
    center_Z = 5 * height_z * np.sin(Alt[k] * np.pi/180)

    theta    = np.linspace(0, 2*np.pi, elements)
    radius   = np.linspace(0, radius, elements)
    z        = (radius ** 2) / (4 * height_z)
    theta_grid, z_grid = np.meshgrid(theta, z)
    x_grid             = [radius[i] * np.cos(theta_grid)[i][:] for i in range(0,elements)]
    y_grid             = [radius[i] * np.sin(theta_grid)[i][:] for i in range(0,elements)]
    x_grid   = np.array(x_grid)
    y_grid   = np.array(y_grid)

    rotation = -rota_alt - 90
    my = np.array([[np.cos(rotation * np.pi/180), 0, np.sin(rotation * np.pi/180)], [0, 1, 0], [-np.sin(rotation * np.pi/180), 0, np.cos(rotation * np.pi/180)]])

    for i in range(0,elements):
        for j in range(0,elements):
            xr = x_grid[i][j] 
            yr = y_grid[i][j] 
            zr = z_grid[i][j] 
            mc = np.array([xr, yr, zr])
            mr = np.dot(my, mc)
            x_grid[i][j] = mr[0]
            y_grid[i][j] = mr[1]
            z_grid[i][j] = mr[2]

    rotation = rota_azi
    mz = np.array([[np.cos(rotation * np.pi/180), -np.sin(rotation * np.pi/180), 0], [np.sin(rotation * np.pi/180), np.cos(rotation * np.pi/180), 0], [0, 0, 1]])

    for i in range(0,elements):
        for j in range(0,elements):
            xr = x_grid[i][j] 
            yr = y_grid[i][j] 
            zr = z_grid[i][j] 
            mc = np.array([xr, yr, zr])
            mr = np.dot(mz, mc)
            x_grid[i][j] = mr[0]
            y_grid[i][j] = mr[1]
            z_grid[i][j] = mr[2]

    x_grid = x_grid + center_x   
    y_grid = y_grid + center_y
    z_grid = z_grid + height_z + center_z

    u, v = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
    x = height_z * np.cos(u) * np.sin(v)
    y = height_z * np.sin(u) * np.sin(v)
    z = height_z * np.cos(v) + height_z 
    a = 5 * height_z  * np.cos(Azi * np.pi/180) * np.cos(Alt * np.pi/180)
    b = 5 * height_z  * np.sin(Azi * np.pi/180) * np.cos(Alt * np.pi/180)
    c = 5 * height_z  * np.sin(Alt * np.pi/180) + height_z
    
    ax4.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r, alpha = 0.25)
    ax4.plot(a, b, c, 'y', lw = 4)
    ax4.plot(0, 0, height_z, 'oc', markersize=10)
    ax4.plot([center_x, center_X], [center_y, center_Y], [height_z + center_z, height_z + center_Z], '--y', lw=2)
    ax4.plot(center_X, center_Y, height_z + center_Z, 'oy', markersize=20)
    ax4.plot_surface(x_grid, y_grid, z_grid, cmap = 'hot', alpha=0.8)
    
    ax4.set_title('Solar Tracker')
    ax4.set_xlabel('Altitud: '+str(np.around(Alt[k], 2)), fontsize=10)
    ax4.set_ylabel('Azimuth: '+str(np.around(Azi[k], 2)), fontsize=10)
    ax4.set_zlabel('Hora: '+str(np.around(hr[k], 2)), fontsize=10)
    ax4.view_init(elev=15, azim=130) 
    set_axes_equal(ax4)
    ax4.xaxis.pane.fill = False # Left pane
    ax4.yaxis.pane.fill = False # Right pane
    ax4.zaxis.pane.fill = False # Right pane
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.set_zticks([])
    
    azimuth.value = str(np.around(Azi[k], 2))
    altitud.value = str(np.around(Alt[k], 2))

i1 = interactive(pdsc, lat = latitud, hou = hora, doy = dia);  

# -------------------------------------------------------------------------------------------------------------------------

# |------------------------------------------------------------------|
# |  Grafica de la radiacion solar incidente y disponible en el dia  |
# |------------------------------------------------------------------|

# Res  = []
# for d in range(1,366):
# # Adjusting day num    
#     day_num = str(d)
#     day_num.rjust(3 + len(day_num), '0')
#     res = str(datetime.strptime("2021" + "-" + day_num, "%Y-%j").strftime("%m-%d-%Y"))
#     Res.append(res)

def srid(lat,hou,doy,msn,lim):
    
    ax3.clear()
    c0  = ((doy - 1) * 360) / 365
    c1  = 229.2 * (0.000075 + 0.001868 * np.cos(c0 * np.pi/180) - 0.032077 * np.sin(c0 * np.pi/180) - 0.014615 * np.cos(2 * c0 * np.pi/180) - 0.04089 * np.sin(2 * c0 * np.pi/180))
    c2  = (4 * 5 + c1) / 60
    w   = 15 * (hou + c2 - 12) # grados
    gam = np.arcsin(0.39795 * np.cos(0.98563 * (doy - 173)  * np.pi/180)) * 180/np.pi
    ws  = np.arccos(-np.tan(gam * np.pi/180) * np.tan(lat * np.pi/180)) * 180/np.pi
    hol = (2 * ws) / 15
    Sns = ws/15 + 12
    Snr = Sns - hol
    sca = 1367 * (1 + 0.033 * np.cos(((360 * doy) / 365) * np.pi/180))
    a0  = 0.98 * (0.4237 - 0.00821 * (6.0 - msn) ** 2)
    a1  = 0.99 * (0.5055 + 0.00595 * (6.5 - msn) ** 2)
    a2  = 1.00 * (0.2711 + 0.01858 * (2.5 - msn) ** 2)
    rb  = a0 + a1 * np.exp(-a2/(np.cos(lat * np.pi/180) * np.cos(gam * np.pi/180) * np.cos(w * np.pi/180) + np.sin(lat * np.pi/180) * np.sin(gam * np.pi/180)))
    rd  = 0.271 - 0.294 * rb
    etr = np.zeros(24) # W/m^2
    gsb = np.zeros(24) # W/m^2
    gsd = np.zeros(24) # W/m^2
    ioh = np.zeros(24) # J/m^2 x h
    x2  = np.arange(0,24)
    for i in x2:
        w1 = 15 * (i + c2 - 12)
        w2 = 15 * (i + 1 + c2 - 12)
        etr[i] = sca * (np.cos(lat * np.pi/180) * np.cos(gam * np.pi/180) * np.cos(w1 * np.pi/180) + np.sin(lat * np.pi/180) * np.sin(gam * np.pi/180))
        gsb[i] = rb * sca * (np.cos(lat * np.pi/180) * np.cos(gam * np.pi/180) * np.cos(w1 * np.pi/180) + np.sin(lat * np.pi/180) * np.sin(gam * np.pi/180))
        gsd[i] = rd * sca * (np.cos(lat * np.pi/180) * np.cos(gam * np.pi/180) * np.cos(w1 * np.pi/180) + np.sin(lat * np.pi/180) * np.sin(gam * np.pi/180))
        ioh[i] = ((12 * 3600) / np.pi) * sca * (np.cos(lat * np.pi/180) * np.cos(gam * np.pi/180) * (np.sin(w2 * np.pi/180) - np.sin(w1 * np.pi/180)) + np.sin(lat * np.pi/180) * np.sin(gam * np.pi/180) * ((w2 - w1) * np.pi/180))
    w3  = 15 * (Snr + c2 - 12)
    w4  = 15 * (Sns + c2 - 12)
    ioa = (((12 * 3600) / np.pi) * sca * (np.cos(lat * np.pi/180) * np.cos(gam * np.pi/180) * (np.sin(w4 * np.pi/180) - np.sin(w3 * np.pi/180)) + np.sin(lat * np.pi/180) * np.sin(gam * np.pi/180) * ((w4 - w3) * np.pi/180)))/3600
    ioa = "%0.2f" % ioa 
    ax3.plot(x2,gsb, color = (1, 1, 1), lw = 2, label = 'Beam Radiation', linestyle = '--')
    ax3.plot(x2,gsd, color = (1, 0, 1), lw = 2, label = 'Diffuse Radiation', linestyle = '--')
    ax3.plot(x2,etr, color = (1, 0, 0), lw = 3.0, label = 'Extraterrestrial Radiation')
    ax3.plot([Snr, Snr],[-1250, 1500], color = (0, 1, 1), lw = 1.5, linestyle = '--')
    ax3.plot([Sns, Sns],[-1250, 1500], color = (0, 1, 1), lw = 1.5, linestyle = '--')
    ax3.bar(np.arange(0.5,24),ioh/3600, color = (1, 1, 0))
    
    li, ls = lim
    w5  = 15 * (li + c2 - 12)
    w6  = 15 * (ls + c2 - 12)
    iod = (((12 * 3600) / np.pi) * sca * (np.cos(lat * np.pi/180) * np.cos(gam * np.pi/180) * (np.sin(w6 * np.pi/180) - np.sin(w5 * np.pi/180)) + np.sin(lat * np.pi/180) * np.sin(gam * np.pi/180) * ((w6 - w5) * np.pi/180)))/3600
    iod = "%0.2f" % iod 
    ax3.plot([li, li],[-1250, 1500], color = (1, 1, 0), lw = 1.5, linestyle = '--')
    ax3.plot([ls, ls],[-1250, 1500], color = (1, 1, 0), lw = 1.5, linestyle = '--')
#     radiacion.value = [str(iod)+r' ${W}/{m^2}$']
    radiacion.value = str(iod)
    
    day_num = str(doy)
    day_num.rjust(3 + len(day_num), '0')
    year = "2021"
    res = datetime.strptime(year + "-" + day_num, "%Y-%j").strftime("%m-%d-%Y")
    
    ax3.text(11.5, -1350, ['Para el dia '+str(res)+' la radiacion disponible total es de '+str(ioa)+r' ${W}/{m^2}$'], fontsize=10, color = (0, 1, 1), horizontalalignment='center')
    ax3.set_xlabel('Horas del dia')
    ax3.set_ylabel('Radiacion')
    ax3.set_title('Solar radiation in this day')
#     plt.xticks(x2)
#     plt.yticks(np.linspace(-1000,1500,6)) 
    ax3.set_xticks(x2)
    ax3.set_yticks(np.linspace(-1000,1500,6))
    ax3.grid(color = 'blue', linestyle = '--', linewidth = 0.5, alpha = 0.8)
    ax3.legend(loc='upper right');
    # plt.show()
i3 = interactive(srid, lat = latitud, doy = dia, hou = hora, msn = altura, lim = horas); 

# -------------------------------------------------------------------------------------------------------------------------

# |--------------------------------------------------------------------------|
# |  Calculo de las entradas y salidas del sol durante todo el presente año  |
# |--------------------------------------------------------------------------|

def sasy(lat,hou,doy):
    
    ax.clear()
    ax.set_title('Sunsets and Sunrises in this year')
    # Calculo de las entradas y salidas del sol durante todo el presente año
    hol  = np.zeros(365)
    sns  = np.zeros(365)
    snr  = np.zeros(365)
    Horr = []
    Hors = []
    Horh = [] 
    
    for d in range(1,366):

    # Declination angle
        gam = np.arcsin(0.39795 * np.cos(0.98563 * (d - 173) * np.pi/180)) * 180/np.pi

    # Time of sunset and sunrise
        ws  = np.arccos(-np.tan(gam  * np.pi/180) * np.tan(lat * np.pi/180)) * 180/np.pi
        hol[d-1] = (2 * ws) / 15
        sns[d-1] = ws/15 + 12
        snr[d-1] = sns[d-1] - hol[d-1]

    # Results    
        snrh = int(snr[d-1])
        snrm = (snr[d-1]*60) % 60
        snrs = (snr[d-1]*3600) % 60
        snsh = int(sns[d-1])
        snsm = (sns[d-1]*60) % 60
        snss = (sns[d-1]*3600) % 60
        holh = int(hol[d-1])
        holm = (hol[d-1]*60) % 60
        hols = (hol[d-1]*3600) % 60
        horr = "%0.0f" % snrh + ':' + "%0.0f" % snrm + ':' + "%0.0f" % snrs
        Horr.append(horr)
        hors = "%0.0f" % snsh + ':' + "%0.0f" % snsm + ':' + "%0.0f" % snss
        Hors.append(hors)
        horh = "%0.0f" % holh + ':' + "%0.0f" % holm + ':' + "%0.0f" % hols
        Horh.append(horh)  
        
    x1  = np.arange(1,366) 
    mnt = int((hou*60) % 60)
    sec = int((hou*3600) % 60)
    
    ax.plot(x1,snr, color = (1, 1, 0), lw = 3)
    ax.plot(x1,sns, color = (1, 1, 0), lw = 3)

    ax.plot(doy,hou,'or', markersize = 10)
    ax.plot(doy,snr[int(doy)],'o', color = (1, 1, 0), markersize = 10)
    ax.plot(doy,sns[int(doy)],'o', color = (1, 1, 0), markersize = 10)
    ax.plot([doy,doy],[0,24],'--r')
    ax.plot([1,365],[hou,hou],'--r')
    ax.text(doy+4, hou+0.5, [str(int(hou))+':'+str(mnt)+':'+str(sec)], fontsize=15)
    
    ax.fill_between(x1, snr, 0, color = (0, 0, 0), alpha = 0.8)
    ax.fill_between(x1, sns, 24, color = (0, 0, 0), alpha = 0.8)
    ax.fill_between(x1, snr, sns, color = (0, 1, 1), alpha = 0.5)
#     ax.set_xlim([1, 365])
#     ax.set_ylim([0, 24])
    plt.xlim([1, 365])
    plt.ylim([0, 24])
    months = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
    plt.xticks([1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335], months, rotation=45, horizontalalignment='left')
    plt.yticks(np.linspace(0,24,25))
#     ax.set_xticks([1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335], months, rotation=45, horizontalalignment='left')
#     ax.set_yticks(np.linspace(0,24,25))
    ax.grid(color = 'blue', linestyle = '--', linewidth = 0.5, alpha = 0.8)
    
    sunrise.value = Horr[int(doy)]
    sunset.value  = Hors[int(doy)]
    duracion.value  = Horh[int(doy)]

i2 = interactive(sasy, lat = latitud, doy = dia, hou = hora); 

# -------------------------------------------------------------------------------------------------------------------------   
    
display(grid1)

GridspecLayout(children=(HTML(value="<center><font size = 6><b><font color='black'>Parabolic Solar Dish Concen…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

GridspecLayout(children=(HTML(value="<center><font size = 3><b><font color='black'>Resultados</b></center>", l…